In [70]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
df = pd.read_csv('flight_df.csv')
df=df.drop(columns=' Flight info')
df=df.drop(columns=' Flight time')


In [71]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

df_encoded= df
df_encoded['Date']=(df_encoded['Date']*100)//100+((df_encoded['Date']*100).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
df_encoded[' Arrival time']=df_encoded[' Arrival time'].str.split(':').str[0].astype(int)
df_encoded[' Departure time']=df_encoded[' Departure time'].str.split(':').str[0].astype(int)
df_encoded[' Price']=df_encoded[' Price'].str.split(',').str[0].astype(int)*1000

df_ohe=enc.fit_transform(df_encoded.drop(columns=' Price')).toarray()


In [72]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = ' Price'
X_train, X_test, y_train, y_test  = train_test_split(
    df_ohe, df_encoded[' Price'], test_size=0.2, random_state=1
)

In [73]:
#LR
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(mean_squared_error(y_pred, y_test, squared = False))
print(r2_score(y_test, y_pred))


109139.80907649004
0.4511619258712718


In [30]:
#OLS

from statsmodels.formula.api import ols
from statsmodels.regression.linear_model import OLS
model = OLS(y_train, X_train)
result = model.fit()
y_pred = result.predict(X_test)
print(r2_score(y_pred, y_test))

-0.10769468220073719


In [69]:
#Experiment

df = pd.read_csv('flight_df.csv')
df=df.drop(columns=' Flight info')
df=df.drop(columns=' Flight time')
#df=df.drop(columns=' Departure time')
df=df.drop(columns=' Arrival time')


####
enc = OneHotEncoder()

df_encoded= df
df_encoded['Date']=(df_encoded['Date']*100)//100+((df_encoded['Date']*100).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
#df_encoded[' Arrival time']=df_encoded[' Arrival time'].str.split(':').str[0].astype(int)
df_encoded[' Departure time']=df_encoded[' Departure time'].str.split(':').str[0].astype(int)
df_encoded[' Price']=df_encoded[' Price'].str.split(',').str[0].astype(int)*1000

df_ohe=enc.fit_transform(df_encoded.drop(columns=' Price')).toarray()

#####
target = ' Price'
X_train, X_test, y_train, y_test  = train_test_split(
    df_ohe, df_encoded[' Price'], test_size=0.2, random_state=1
)


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(r2_score(y_test, y_pred))


0.43144437486062814


In [ ]:
#Arrival 빼고: 0.43144437486062814
#Departure 빼고: 0.410816052709579